# Connecting Chameleon Experiements with AWS Direct Connect using Internet2 CloudConnect

## Getting started

**NOTE:**: AWS Direct Connect functionality is in early access beta, meaning it currently requires a more hands-on approach and some manual steps. Assistance from the Chameleon team is likely required, especially if you do not have access to Internet2's Cloud Connect portal. Please reach out if you're interested in working on a setup like this.

Now it is possible to create a network that stitches Chameleon to commercial clouds, such as AWS. 

This document shows the steps to establish stitchable layer 2 connection from Chameleon to AWS using [Internet2 CloudConnect](https://www.internet2.edu/blogs/detail/15119)

Test setup is created across Chameleon Cloud and AWS. This setup can also be created on ExoGENI sites or any other edge sites that have access to any [ExoGENI Stitchports](https://wiki.exogeni.net/doku.php?id=public:experimenters:resource_types:start#stitch_port_identifiers). 

## Topology

<br>
<center>
    <img src="./CloudConnect-Figs/Pre-built-AWS-Chameleon.png"><br>
    <em>Pre-built AWS infrastructure and Chameleon Cloud sites</em>
</center>
<br>

<br>
<center>
    <img src="./CloudConnect-Figs/layer-3-connection.png"><br>
    <em>Network layout for layer-3 connection</em>
</center>
<br>

## AWS Infrastructure
AWS-side entities include VPC, Virtual Private Gateway, DirectConnect Gateway, Virtual Interface (dxvif) on the DirectConnect Gateway, Layer-3 peering (dxpeer)  and a Connection (dxcon). DirectConnect Gateway is associated with Virtual Private Gateway (vgw). 

Details of the VPC creation procedure can be found on [RENCI-NRIG/exogeni](https://github.com/RENCI-NRIG/exogeni/blob/master/infrastructure/aws/aws-vpc-create.md) repo. 
Same VPC creation procedure is also described on a separate document in Chameleon Google Drive 
for Chameleon Team. 
Chameleon Google Drive → ExoGENI Stitching → [Stitching to Public Clouds](https://docs.google.com/document/d/1XT0gA_akMARyxZxi9Epb3YCssQbnlVLwyf31_YT2P4w/edit#heading=h.ey1vfu2c25nq) : “AWS Components” 

## Chameleon Cloud Infrastructure
Steps below require OpenStack and Blazar clients on the local computer. 

### Notes

1. Some of the resources will be associated with the user who is running this doc, in this case `$USER`.
1. If you are only a member of one project, you can skip this step, as the project will be selected for you by default. Otherwise, take a look at the current value of the environmental variable `OS_PROJECT_NAME`:

In [ ]:
# Setting the project name
export OS_PROJECT_NAME='CH-816532' #Set to your project name

First, let's test the connection by asking for a list of ssh keypairs:

In [ ]:
openstack keypair list
export KEY_NAME='pruth-jupyter' #Set to your key name

If the output looks like this:
```
+------------+-------------------------------------------------+
| Name       | Fingerprint                                     |
+------------+-------------------------------------------------+
| defaultkey | d0:89:5b:61:6a:64:dd:c8:db:67:32:32:45:71:b0:b8 |
+------------+-------------------------------------------------+
```
you can continue to the next step. However, if it looks something like this:
```
The request you have made requires authentication. (HTTP 401) (Request-ID: req-76ad404f-0043-45e9-84cf-0504843888ab)
```
figure out what is going on before continuing. One thing to check is whether the project name you provided works (the nickname issue mentioned before is one possible reason).

You can also set the site you want to use via the `OS_REGION_NAME` setting; this defaults to `CHI@UC`.

In [ ]:
# Set region (Optional, default to 'CHI@UC')
export OS_REGION_NAME='CHI@UC'
echo $OS_REGION_NAME

### Create neutron networks
Create leases for network resources. 

When leases are created and they are active, networks will be created automatically. 

Network-1 is used for stitching to AWS. It is reserved on “exogeni” provider.
Network-2 is used for management (external access to the instance). It is optional and existing networks (eg. sharednet1, sharedwan1) can be used for this purpose.

#### Creating network 1:

In [ ]:
#
# Create leases for networks on Chameleon Cloud
#


LEASE_TYPE="network"

TASK="cloudconnect"
LEASE_PREFIX="${USER}-${TASK}-${LEASE_TYPE}"

OF_CONTROLLER_IP="147.72.248.27"
OF_CONTROLLER_PORT="6653"

VFC_NAME_1="cloudconnect1"
PROVIDER_1="exogeni"
NET_NAME_1="${USER}-net-${PROVIDER_1}-${VFC_NAME_1}-1"
       
# Create Lease for network-1 on exogeni provider

VFC_NAME="${VFC_NAME_1}"
PROVIDER="${PROVIDER_1}"
NET_NAME="${NET_NAME_1}"

LEASE_NAME="${LEASE_PREFIX}-${PROVIDER}-1"


# Possible options for network creation on Chameleon Cloud
# 1 - custom openflow controller on named vfc
# 2 - stock openflow controller on named vfc
# 3 - stock openflow controller without named vfc

# Option 1
#blazar lease-create \
#   --reservation resource_type=network,network_name=${NET_NAME},network_description="OFController=${OF_CONTROLLER_IP}:${OF_CONTROLLER_PORT},VSwitchName=${VFC_NAME}",resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
#   ${LEASE_NAME}

# Option 2
blazar lease-create \
   --reservation resource_type=network,network_name=${NET_NAME},network_description="VSwitchName=${VFC_NAME}",resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
   ${LEASE_NAME}

# Option 3
#blazar lease-create \
#   --reservation resource_type=network,network_name=${NET_NAME},resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
#   ${LEASE_NAME}


#### Creating network 2:

In [ ]:
#
# Create leases for networks on Chameleon Cloud
#

LEASE_TYPE="network"

TASK="cloudconnect"
LEASE_PREFIX="${USER}-${TASK}-${LEASE_TYPE}"

OF_CONTROLLER_IP="147.72.248.27"
OF_CONTROLLER_PORT="6653"

VFC_NAME_1="cloudconnect2"
PROVIDER_1="physnet1"
NET_NAME_1="${USER}-net-${PROVIDER_1}-${VFC_NAME_1}-1"

VFC_NAME="${VFC_NAME_1}"
PROVIDER="${PROVIDER_1}"
NET_NAME="${NET_NAME_1}"

LEASE_NAME="${LEASE_PREFIX}-${PROVIDER}-1"


# Possible options for network creation on Chameleon Cloud
# 1 - custom openflow controller on named vfc
# 2 - stock openflow controller on named vfc
# 3 - stock openflow controller without named vfc

# Option 1
#blazar lease-create \
#   --reservation resource_type=network,network_name=${NET_NAME},network_description="OFController=${OF_CONTROLLER_IP}:${OF_CONTROLLER_PORT},VSwitchName=${VFC_NAME}",resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
#   ${LEASE_NAME}

# Option 2
blazar lease-create \
   --reservation resource_type=network,network_name=${NET_NAME},network_description="VSwitchName=${VFC_NAME}",resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
   ${LEASE_NAME}

# Option 3
#blazar lease-create \
#   --reservation resource_type=network,network_name=${NET_NAME},resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
#   ${LEASE_NAME}


After the leases are active and networks are created, create subnets.

#### Create subnet 1:

In [ ]:
#
# Create subnet 
#

VFC_NAME_1="cloudconnect1"
PROVIDER_1="exogeni"
NET_NAME_1="${USER}-net-${PROVIDER_1}-${VFC_NAME_1}-1"

# Create subnet on network-1 

VFC_NAME="${VFC_NAME_1}"
PROVIDER="${PROVIDER_1}"
NET_NAME="${NET_NAME_1}"

SUBNET_NAME="subnet-${NET_NAME}"
DNS_NAMESERVER1="130.202.101.6"
DNS_NAMESERVER2="130.202.101.37"
 
CIDR="172.16.200.0/24"
GATEWAY_IP="172.16.200.127"
DHCP_ALLOCATION_START="172.16.200.11"
DHCP_ALLOCATION_END="172.16.200.99"


openstack subnet create --subnet-range ${CIDR} \
                   --no-dhcp \
                   --gateway ${GATEWAY_IP} \
                   --network ${NET_NAME} ${SUBNET_NAME}

#### Create subnet 2:

In [ ]:
#
# Create subnet 
#

VFC_NAME_1="cloudconnect2"
PROVIDER_1="physnet1"
NET_NAME_1="${USER}-net-${PROVIDER_1}-${VFC_NAME_1}-1"

# Create subnet on network-2 

VFC_NAME="${VFC_NAME_1}"
PROVIDER="${PROVIDER_1}"
NET_NAME="${NET_NAME_1}"

#VLAN_TAG=$(openstack network show ${NET_NAME} | grep segmentation_id | awk '{print $4}')
 
SUBNET_NAME="subnet-${NET_NAME}"
DNS_NAMESERVER1="130.202.101.6"
DNS_NAMESERVER2="130.202.101.37"
 

CIDR="192.168.200.0/24"
GATEWAY_IP="192.168.200.254"
DHCP_ALLOCATION_START="192.168.200.51"
DHCP_ALLOCATION_END="192.168.200.90"

openstack subnet create --subnet-range ${CIDR} \
                   --dhcp \
                   --allocation-pool start=${DHCP_ALLOCATION_START},end=${DHCP_ALLOCATION_END} \
                   --dns-nameserver ${DNS_NAMESERVER1} \
                   --dns-nameserver ${DNS_NAMESERVER2} \
                   --gateway ${GATEWAY_IP} \
                   --network ${NET_NAME} ${SUBNET_NAME}

# Create router for the subnet

ROUTER_NAME="router-${NET_NAME}"
EXTERNAL_NET="public"

openstack router create ${ROUTER_NAME}
openstack router add subnet ${ROUTER_NAME} ${SUBNET_NAME} 
openstack router set --external-gateway ${EXTERNAL_NET} ${ROUTER_NAME}

#### Create a lease for floating IP addresses.

In [ ]:
# 
# Create lease for floating ips
#

LEASE_TYPE="floatingip"

TASK="cloudconnect"
LEASE_PREFIX="${USER}-${TASK}-${LEASE_TYPE}"

LEASE_NAME="${LEASE_PREFIX}"

AMOUNT="3"

PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

blazar lease-create \
  --reservation "resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=${AMOUNT}" \
  "$LEASE_NAME"


### Create instances
In this setup, 2 instances are created. 
	BGP speaker (sw-bgp) is an instance that is built on a dual-nic Haswell node.
Regular node (sw-instance-1) in the subnet can be built on a Skylake or Haswell node.

Create a lease with 2-3 dual-nic Haswell nodes. 


In [ ]:
# Create Lease for servers

LEASE_TYPE="server"

NODE_TYPE_1="compute_skylake"
NODE_TYPE_2="compute_haswell"

TASK="cloudconnect"
LEASE_PREFIX="${USER}-${TASK}-${LEASE_TYPE}"

MIN=2
MAX=3


#
# Haswell Nodes with dual-NICs enabled
#

NODE_TYPE=${NODE_TYPE_2}
LEASE_NAME="${LEASE_PREFIX}-${NODE_TYPE}-1"


# Create server lease with dual-nic nodes

blazar lease-create \
      --physical-reservation min=${MIN},max=${MAX},resource_properties="[\"and\",[\"==\",\"\$network_adapters.1.enabled\",\"True\"],[\"==\",\"\$node_type\",\"compute_haswell\"]]" \
      ${LEASE_NAME}

# Get the reservation ID

RESERVATION_ID=$(blazar lease-show  -f json ${LEASE_NAME} | jq -r .reservations | jq -r .id)
LEASE_ID=$(blazar lease-show  -f json ${LEASE_NAME} | jq -r .id)

# Get nodes from the reservation

lease=$LEASE_ID
hosts=$(blazar host-list -f json | jq 'map({"key": .id, "value": .hypervisor_hostname}) | from_entries')
NODES=$(blazar host-allocation-list -f json  \
        | jq -r --argjson hosts "$hosts" \
        "map(select(.reservations[]|select(.lease_id == \"$lease\"))) | map(\$hosts[.resource_id])[] ")

HASWELL_NODES=( $NODES )
RESERVATION_ID_HASWELL=$RESERVATION_ID

#### Create instances (sw-bgp and sw-instance-1).
For now we use the Haswell nodes.

In [ ]:
LEASE_TYPE="server"

NODE_TYPE_1="compute_skylake"
NODE_TYPE_2="compute_haswell"

TASK="cloudconnect"
LEASE_PREFIX="${USER}-${TASK}-${LEASE_TYPE}"

# Skylake Node Lease

NODE_TYPE=${NODE_TYPE_1}
LEASE_NAME_SKYLAKE="${LEASE_PREFIX}-${NODE_TYPE}-1"

# Haswell Node Lease

NODE_TYPE=${NODE_TYPE_2}
LEASE_NAME_HASWELL="${LEASE_PREFIX}-${NODE_TYPE}-1"

# Haswell - Get the reservation ID

LEASE_NAME=${LEASE_NAME_HASWELL}

RESERVATION_ID=$(blazar lease-show  -f json ${LEASE_NAME} | jq -r .reservations | jq -r .id)
LEASE_ID=$(blazar lease-show  -f json ${LEASE_NAME} | jq -r .id)

# Haswell - Get nodes from the reservation

hosts=$(blazar host-list -f json | jq 'map({"key": .id, "value": .hypervisor_hostname}) | from_entries')
NODES=$(blazar host-allocation-list -f json  \
        | jq -r --argjson hosts "$hosts" \
        "map(select(.reservations[]|select(.lease_id == \"$LEASE_ID\"))) | map(\$hosts[.resource_id])[] ")

HASWELL_NODES=( $NODES )
RESERVATION_ID_HASWELL=$RESERVATION_ID

HASWELL_1="${HASWELL_NODES[0]}"
HASWELL_2="${HASWELL_NODES[1]}"
HASWELL_3="${HASWELL_NODES[2]}"

# Get network UUIDs
VFC_NAME_1="cloudconnect1"
PROVIDER_1="exogeni"
NET_NAME_1="${USER}-net-${PROVIDER_1}-${VFC_NAME_1}-1"
NET_UUID_1=$( openstack network show -f value -c id ${NET_NAME_1} )

VFC_NAME_2="cloudconnect2"
PROVIDER_2="physnet1"
NET_NAME_2="${USER}-net-${PROVIDER_2}-${VFC_NAME_2}-1"
NET_UUID_2=$( openstack network show -f value -c id ${NET_NAME_2} )

INSTANCE_1="sw-bgp"
INSTANCE_2="sw-instance-1"

NET_1_IP_1="172.16.200.1"
NET_1_IP_2="172.16.200.2"
NET_1_IP_3="172.16.200.3"

NET_2_IP_1="192.168.200.61"
NET_2_IP_2="192.168.200.62"
NET_2_IP_3="192.168.200.63"



FLAVOR="baremetal"
IMAGE_CENTOS="CC-CentOS7"
KEY="renci-key"
SEC_GROUP="default"

IMAGE=${IMAGE_CENTOS}
USER_DATA_FILE="postbootscript.sh"

#
# Create instance BGP speaker on dual-nic Haswell node 
#
SERVER=${INSTANCE_1}
PHYSICAL=${HASWELL_1}
IP_ADDR_1=${NET_1_IP_1}
IP_ADDR_2=${NET_2_IP_1}
RESERVATION_ID=${RESERVATION_ID_HASWELL}

openstack server create \
  --image ${IMAGE} \
  --flavor ${FLAVOR} \
  --key-name ${KEY} \
  --nic net-id=${NET_UUID_2},v4-fixed-ip=${IP_ADDR_2} \
  --nic net-id=${NET_UUID_1},v4-fixed-ip=${IP_ADDR_1} \
  --hint reservation=${RESERVATION_ID} \
  --hint query='["=","$hypervisor_hostname","$PHYSICAL"]' \
  ${SERVER}

sleep 3;

#
# Create regular instance on Haswell node 
#
SERVER=${INSTANCE_2}
PHYSICAL=${HASWELL_3}
IP_ADDR_2=${NET_2_IP_2}
RESERVATION_ID=${RESERVATION_ID_HASWELL}

openstack server create \
  --image ${IMAGE} \
  --flavor ${FLAVOR} \
  --key-name ${KEY} \
  --nic net-id=${NET_UUID_2},v4-fixed-ip=${IP_ADDR_2} \
  --hint reservation=${RESERVATION_ID} \
  --hint query='["=","$hypervisor_hostname","$PHYSICAL"]' \
  ${SERVER}

#### Associate floating IPs with instances.
Note: Please run this part after your instance is up and running. 

This step may not work with this Jupyter interface. Please go to Chameleon web portal to manually associate floating IPs with your instances if the following commands fail.

In [ ]:
SERVER_1="sw-bgp"
SERVER_2="sw-instance-1" 

# Update the following according to the floating IPs in your network -> Floating IPs section
FLOATING_1="192.5.87.227"
FLOATING_2="192.5.87.208" 

openstack server add floating ip ${SERVER_1} ${FLOATING_1}
openstack server add floating ip ${SERVER_2} ${FLOATING_2}


### Run BGP instance on Chameleon Cloud
Once the nodes finished spawning, login to the BGP instance (sw-bgp) and assign IP address to the interface that is facing AWS.

```
ip addr add 172.16.100.1/30 dev eno2
```

Clone exogeni-recipes/chameleon repo for installation script.

```
yum install -y git
mkdir ~/exogeni-recipes
cd ~/exogeni-recipes/
git init
git config core.sparsecheckout true
echo chameleon/cloudconnect/cloudconnect_install_bgp.sh >> .git/info/sparse-checkout
git remote add -f origin https://mcevik0@github.com/RENCI-NRIG/exogeni-recipes.git
git pull origin master
```

Run installation script to create the Quagga Docker (CRA) container. 
This script configures Quagga for BGP. Variables defined in the script have to be adjusted for the Layer-3 topology.

```
cd ~/exogeni-recipes/chameleon/cloudconnect
BGP_AUTH_KEY="BgpAuthPass"
./cloudconnect_install_bgp.sh $BGP_AUTH_KEY
```

Current parameters for the test setup is below.

```
# Modify /root/corsa_cra/corsa_cra/corsa_cra/docker/corsa_cra/quagga/bgpd.conf
export LOCAL_ASN="65016"
export LOCAL_ROUTER_IP="172.16.100.1"
export LOCAL_SUBNET="192.168.200.0\/24"
export REMOTE_ROUTER_IP="172.16.100.2"
export REMOTE_ASN="55038"
export REMOTE_DESC="AWS"
export BGP_PASSWORD="BgpAuthPass"

# Modify  /root/corsa_cra/corsa_cra/corsa_cra/docker/corsa_cra/quagga/zebra.conf
export INTERFACE_FACING_AWS="eno1"
export INTERFACE_FACING_LOCAL="eno2"
export IP_ADDRESS_FACING_AWS="172.16.100.1\/24"
export IP_ADDRESS_FACING_LOCAL="192.168.200.61\/24"
```

Check the status of Quagga (CRA) container and BGP. You should see one entry from `show ip bgp`, at this time. After the connection to AWS is enabled, there should be four entries.

```
[root@sw-bgp ~]# docker ps -a
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
4b1ff8a09f0c        cra_2               "/bin/sh -c '/start_…"   5 hours ago         Up 5 hours                              cra_2

[root@sw-bgp ~]# docker attach cra_2

root@sw-bgp:/# vtysh 

Hello, this is Quagga (version 1.0.20160315-ci.NetDEF.org-20160610.130007-git.5f6788i).
Copyright 1996-2005 Kunihiro Ishiguro, et al.

cra-router# show ip bgp 
BGP table version is 0, local router ID is 172.16.100.1
Status codes: s suppressed, d damped, h history, * valid, > best, = multipath,
              i internal, r RIB-failure, S Stale, R Removed
Origin codes: i - IGP, e - EGP, ? - incomplete
   Network          Next Hop            Metric LocPrf Weight Path
*> 192.168.200.0    0.0.0.0                  0          32768 i
Displayed  1 out of 1 total prefixes

```

#### Configure BGP instance for routing
Kernel parameters are applied to configure forwarding as well as TCP tuning for throughput.

```
cat << EOF >> /etc/sysctl.conf
net.core.rmem_default = 16777216
net.core.wmem_default = 16777216
net.core.netdev_max_backlog=250000
net.core.rmem_max = 536870912
net.core.wmem_max = 536870912
net.ipv4.conf.all.arp_announce=2
net.ipv4.conf.all.arp_filter=1
net.ipv4.conf.all.arp_ignore=1
net.ipv4.conf.default.arp_filter=1
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_no_metrics_save = 1
net.ipv4.tcp_rmem = 4096 87380 268435456
net.ipv4.tcp_wmem = 4096 65536 268435456
net.ipv4.tcp_mtu_probing=1
net.core.netdev_budget = 600
net.core.default_qdisc = fq
net.ipv4.ip_forward = 1
EOF

sysctl -p 
```

Configure iptables to enable forwarding.

```
IF_NET1="eno1"
IF_NET2="eno2"

for INTERFACE in ${IF_NET1} ${IF_NET2}; do
    ip link set ${INTERFACE} mtu 9000
    ip link set ${INTERFACE} txqueuelen 10000
done

iptables -I FORWARD -i ${IF_NET2} -o ${IF_NET1} -m state --state NEW,ESTABLISHED,RELATED -j ACCEPT
iptables -I FORWARD -i ${IF_NET1} -o ${IF_NET2} -m state --state NEW,ESTABLISHED,RELATED -j ACCEPT 
```

#### Add routes to AWS on regular instances
Instances connected to the local subnet connect to AWS through the BGP speaker.

Current routes on the regular node:

```
[root@sw-instance-1 ~]# ip route
default via 192.168.200.254 dev eno1 
169.254.0.0/16 dev eno1 scope link metric 1002 
169.254.169.254 via 192.168.200.51 dev eno1 proto static 
192.168.200.0/24 dev eno1 proto kernel scope link src 192.168.200.62 
```

Add route for AWS:

```
[root@sw-instance-1 ~]# ip route add 10.12.0.0/16 via 192.168.200.61 

[root@sw-instance-1 ~]# ip route
default via 192.168.200.254 dev eno1 
10.12.0.0/16 via 192.168.200.61 dev eno1 
169.254.0.0/16 dev eno1 scope link metric 1002 
169.254.169.254 via 192.168.200.51 dev eno1 proto static 
192.168.200.0/24 dev eno1 proto kernel scope link src 192.168.200.62 
```

#### Internet2 CloudConnect Layer-3 Provisioning
Note: It is very likely that you do not have CloudConnect access. The Chameleon team can support you in creating the required entities/endpoints on CloudConnect for these steps, if need be.

Please see [this document](https://docs.google.com/document/d/1tW34zkIEB_QmhplcbJJh8HnMapdouxNq2cq0GojJLFk/edit?usp=sharing) for additional details on what's going on behind the scene.

#### After Internet2 CloudConnect Layer-3 Provisioning

### Set up AWS Direct Connect Connection
#### Connection
On the AWS side, we have a new connection created for us by CloudConnect.

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD.png"><br>
    <em>AWS Direct Connect.</em>
</center>
<br>

Select and Accept the connection and it will change state into “available”.

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-ordering.png"><br>
    <em>AWS Direct Connect in ordering status.</em>
</center>
<br>

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-available.png"><br>
    <em>AWS Direct Connect is available.</em>
</center>
<br>

#### Direct Connect Gateway
Create Direct Connect Gateway under “Direct Connect Gateways” on the left pane. 

Enter the ASN that is used for US East (N.Virgina) - Hosted Connect entity peering to the “Amazon side ASN” field (65016 in this setup). 

Direct Connect Gateway will be created.

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-create-gateway.png"><br>
    <em>AWS Direct Connect Gateway is being created.</em>
</center>
<br>

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-gateway-created.png"><br>
    <em>AWS Direct Connect Gateway has been created.</em>
</center>
<br>

#### Virtual Interface
Create Virtual Interface on the Direct Connect Gateway.
<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-virtual-interface.png"><br>
    <em>AWS Direct Connect Virtual Interface.</em>
</center>
<br>

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-virtual-interface-config.png"><br>
    <em>AWS Direct Connect Virtual Interface configuration.</em>
</center>
<br>

Virtual Interface will be attached to the Direct Connect Gateway.

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-virtual-interface-attach.png"><br>
    <em>AWS Direct Connect Virtual Interface attached.</em>
</center>
<br>

On the Virtual Interface information, it will show up as “down”. 

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-virtual-interface-status.png"><br>
    <em>AWS Direct Connect Virtual Interface showing status as "down".</em>
</center>
<br>

Now we need to go back to CloudConnect to modify the peering with the information that is seen above. <b>This step most likely needs to be performed by a Chameleon team member.</b> Please see [this document](https://docs.google.com/document/d/1tW34zkIEB_QmhplcbJJh8HnMapdouxNq2cq0GojJLFk/edit?usp=sharing) for additional details on what's going on behind the scene.


### AWS | VPC - Direct Connect Association
#### AWS VPC and Virtual Private Gateway
Details of the VPC creation procedure can be found on [RENCI-NRIG/exogeni](https://github.com/RENCI-NRIG/exogeni/blob/master/infrastructure/aws/aws-vpc-create.md) repo. 
Same VPC creation procedure is also described on a separate document in Chameleon Google Drive 
for Chameleon Team. 

#### Associate Virtual Private Gateway with Direct Connect Gateway
On the Direct Connect Gateways page, select the Direct Connect Gateway.

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-gateway-association.png"><br>
    <em>AWS Direct Connect associate gateway.</em>
</center>
<br>
Under Gateway Associations Tab, Associate Gateway.

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-gateway-associated.png"><br>
    <em>AWS Direct Connect gateway associated.</em>
</center>
<br>

Now log back in to the Chameleon sw-bgp node, AWS routes will start to show up.
```
cra-router# show ip bgp 
BGP table version is 0, local router ID is 172.16.100.1
Status codes: s suppressed, d damped, h history, * valid, > best, = multipath,
              i internal, r RIB-failure, S Stale, R Removed
Origin codes: i - IGP, e - EGP, ? - incomplete

   Network          Next Hop            Metric LocPrf Weight Path
*> 10.12.0.0/16     172.16.100.2                           0 55038 65016 i
*> 169.254.255.20/30
                    172.16.100.2                           0 55038 i
*> 172.16.100.0/24  172.16.100.2                           0 55038 i
*> 192.168.200.0    0.0.0.0                  0          32768 i

Displayed  4 out of 4 total prefixes
```

### AWS | Instance Creation
Launch instance in chameleon-vpc.
<br>
<center>
    <img src="./CloudConnect-Figs/AWS-DD-launch-instance-vpc.png"><br>
    <em>Launch instance in chameleon-vpc.</em>
</center>
<br>

To allow inbound ICMP traffic, modify the security rules.
<br>
<center>
    <img src="./CloudConnect-Figs/AWS-traffic-rules.png"><br>
    <em>Modify the security rules.</em>
</center>
<br>

Now we should be able to ping from the Chameleon sw-instance-1 node to the AWS node. Ping test below.
```
[root@sw-instance-1 ~]# ping 10.12.1.219 -M do -s 8972 -c 5
PING 10.12.1.219 (10.12.1.219) 8972(9000) bytes of data.
8980 bytes from 10.12.1.219: icmp_seq=1 ttl=55 time=18.6 ms
8980 bytes from 10.12.1.219: icmp_seq=2 ttl=55 time=18.6 ms
8980 bytes from 10.12.1.219: icmp_seq=3 ttl=55 time=18.6 ms
8980 bytes from 10.12.1.219: icmp_seq=4 ttl=55 time=18.6 ms
8980 bytes from 10.12.1.219: icmp_seq=5 ttl=55 time=18.6 ms

--- 10.12.1.219 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4007ms
rtt min/avg/max/mdev = 18.677/18.684/18.696/0.122 ms

[root@ip-10-12-1-219 ~]# ping 192.168.200.62 -M do -s 8972 -c 5
PING 192.168.200.62 (192.168.200.62) 8972(9000) bytes of data.
8980 bytes from 192.168.200.62: icmp_seq=1 ttl=56 time=18.6 ms
8980 bytes from 192.168.200.62: icmp_seq=2 ttl=56 time=18.6 ms
8980 bytes from 192.168.200.62: icmp_seq=3 ttl=56 time=18.7 ms
8980 bytes from 192.168.200.62: icmp_seq=4 ttl=56 time=18.7 ms
8980 bytes from 192.168.200.62: icmp_seq=5 ttl=56 time=18.7 ms

--- 192.168.200.62 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 11ms
rtt min/avg/max/mdev = 18.604/18.650/18.688/0.093 ms
```


### Tear down
#### AWS | Direct Connect - Disassociate Virtual Private Gateway

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-disassociate-gateway.png"><br>
    <em>Disassociate Virtual Private Gateway.</em>
</center>
<br>

#### AWS | VPC - Detach Virtual Private Gateway

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-detach-gateway.png"><br>
    <em>Detach Virtual Private Gateway.</em>
</center>
<br>

#### AWS | Direct Connect - Delete Virtual Interface

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-delete-virtual-interface.png"><br>
    <em>Delete Virtual Interface.</em>
</center>
<br>

#### Finally, let the Chameleon admin know when you would like to tear down the connection. We will delete the connection on CloudConnect side.